In [3]:
from flask import Flask, render_template, request, redirect

import requests
import simplejson as json
import pandas as pd 
import numpy as np 
from bokeh.layouts import row, gridplot
from bokeh.plotting import figure, show, save, output_file
from bokeh.embed import components
from shutil import copyfile
import matplotlib.pyplot as plt  
import os    
from pandas.io.json import json_normalize
from bokeh.io import push_notebook, show, output_notebook



def get_stock_info(ticker):
    api_key = '5eWyXUgyDyNbYXctVW53'  
    start_date = '2013-01-01'    
    end_date = '2017-05-11'      
    ticker = 'AaPL'                
    api_url = "https://www.quandl.com/api/v3/datasets/WIKI/%s/data.json" % ticker
    payload = {'api_key': api_key, 'start_date': start_date, 'end_date': end_date, 'order': 'asc'}
    r = requests.get(api_url, params = payload)
    r_raw = r.json()   
    json_normalize(r_raw)    
    r_cols = r_raw['dataset_data']['column_names'] 
    r_data = r_raw['dataset_data']['data']   

    numdays = 3   
    df = pd.DataFrame(r_data)
    df.columns = r_cols
    return df
df = get_stock_info('FB')
df

,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2013-01-02,553.820,555.0000,541.6300,549.0300,20018500.0,0.00,1.0,71.816894,71.969911,70.236149,71.195748,140129500.0
1,2013-01-03,547.880,549.6700,541.0000,542.0959,12605900.0,0.00,1.0,71.046621,71.278740,70.154453,70.296565,88241300.0
2,2013-01-04,536.965,538.6299,525.8286,527.0000,21226200.0,0.00,1.0,69.631213,69.847109,68.187094,68.338996,148583400.0
3,2013-01-07,522.000,529.3000,515.2000,523.9000,17291300.0,0.00,1.0,67.690619,68.637250,66.808825,67.937002,121039100.0
4,2013-01-08,529.210,531.8900,521.2500,525.3100,16382400.0,0.00,1.0,68.625579,68.973109,67.593362,68.119845,114676800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,2017-05-05,146.760,148.9800,146.7600,148.9600,26787359.0,0.00,1.0,145.571223,147.773241,145.571223,147.753403,26787359.0
1094,2017-05-08,149.030,153.7000,149.0300,153.0000,48339210.0,0.00,1.0,147.822836,152.455008,147.822836,151.760678,48339210.0
1095,2017-05-09,153.870,154.8800,153.4500,153.9600,35942435.0,0.00,1.0,152.623631,153.625450,152.207033,152.712902,35942435.0
1096,2017-05-10,153.630,153.9400,152.1100,153.2600,25670456.0,0.00,1.0,152.385575,152.693064,150.877887,152.018572,25670456.0


In [4]:
def datetime(x):
    return np.array(x, dtype=np.datetime64)

def output_plot(df, ticker, closing_pr, adj_closing_pr, opening_pr, adj_opening_pr):
    closing_pr = True   #debug, can be removed
    adj_closing_pr = True  #debug, can be removed
    opening_pr = False    #debug, can be removed
    adj_opening_pr = False  #debug, can be removed

    output_notebook()
    p1 = figure(x_axis_type="datetime", title="Data from Quandle WIKI set")
    p1.grid.grid_line_alpha=0.3
    p1.xaxis.axis_label = 'Date'
    p1.yaxis.axis_label = 'Price'

    if opening_pr:
        p1.line(datetime(df['Date']), df['Open'], color='#A6CEE3', legend= ticker+':Open')
    if adj_opening_pr:
        p1.line(datetime(df['Date']), df['Adj. Open'], color='#B2DF8A', legend= ticker+':Adj. Open')
    if closing_pr:
        p1.line(datetime(df['Date']), df['Close'], color='#33A02C', legend=ticker+':Close')
    if adj_closing_pr:
        p1.line(datetime(df['Date']), df['Adj. Close'], color='#FB9A99', legend=ticker+': Adj. Close')
    p1.legend.location = "top_left"

    output_file("stocks.html", title="Stock_Info", mode='relative', root_dir= '/templates')
    #copyfile("stocks.html", "templates/")
    script, div = components(p1)     # returns necessary components to plot 
    save(gridplot([[p1]], plot_width=400, plot_height=400))  # open a browser
    copyfile("stocks.html", "templates/stocks.html")
    return (script, div)

In [7]:
ticker = 'fb'
script, div = output_plot(df, ticker, True, True, False, False)

Loading BokehJS ...

In [ ]:
app = Flask(__name__)

@app.route('/')
def main():
    return redirect('/index')

@app.route('/index',methods=['GET','POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    else:
        #request was a POST
        #ticker = request.form['ticker']
        #closing_pr = request.form['closing_pr']
        #adj_closing_pr = request.form['adj_closing_pr']
        #opening_pr = request.form['opening_pr']
        #adj_opening_pr = request.form['adj_opening_pr']
        return redirect('/main')
    
@app.route('/main')
def main_plot():
    df = get_stock_info('FB')
    script, div = output_plot(df, 'FB', True, False, True, False)
    return render_template(html_out)  # added script, div
    #return 'Hello World..'   
    
if __name__ == '__main__':
    app.run(host='0.0.0.0')


In [ ]:
copyfile("stocks.html", "templates/")